##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Play with AI - Guess the word

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to play "Guess the word" game.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Guess_the_word.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [1]:
import os
from google.colab import userdata

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies

Install Keras and KerasNLP.

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

### Create a chat helper to manage the conversation state

In [ ]:
import re

import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.compile(sampler="top_k")


class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

  def show_history(self):
    for h in self.history:
      print(h)


chat = ChatState(gemma_lm)

## Play the game

In [4]:
theme = input("Choose your theme: ")
setup_message = f"Generate a random single word from {theme}."

chat.history.clear()
answer = chat.send_message(setup_message).split()[0]
answer = re.sub(r"\W+", "", answer)  # excludes all numbers, letters and '_'
chat.history.clear()
cmd_exit = "quit"
question = f'Describe the word "{answer}" without saying it.'

resp = ""
while resp.lower() != answer.lower() and resp != cmd_exit:
    text = chat.send_message(question)
    if resp == "":
        print(f'Guess what I\'m thinking.\nType "{cmd_exit}" if you want to quit.')
    remove_answer = re.compile(re.escape(answer), re.IGNORECASE)
    text = remove_answer.sub("XXXX", text)
    print(text)
    resp = input("\n> ")

if resp == cmd_exit:
    print(f"The answer was {answer}.\n")
else:
    print("Correct!")

Choose your theme: animal
Guess what I'm thinking.
Type "quit" if you want to quit.
A playful, furry swimmer. Known for its playful antics and clever use of tools.  A member of the weasel-like family with a distinctive, waterproof coat. 
<end_of_turn>

> platypus
A creature that spends its days by a river or lake, often seen splashing and diving with its distinctive, thick fur.  It's known for its playful demeanor and ability to hold a surprisingly large amount of water in its paws. 
<end_of_turn>

> beaver
A small, aquatic mammal with a playful, curious nature, often spotted near water, known for its distinctive, waterproof fur and love for swimming. 
<end_of_turn>

> otter
Correct!
